# S1 CU sample H2 ads isotherm 

In [67]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from sklearn.metrics import r2_score
import seaborn
seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 

In [68]:
#reading the Emission current, pressures
vaclog=pd.read_csv("cooldown vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
#making the column into a datetime object
vac_timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(vac_timestamps-vac_timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
#drop index column (not necessary)
# vaclog.drop(index=vaclog.index[0], 
#         axis=0, 
#         inplace=True)
vaclog["Time"] = vac_timestamps.dt.strftime('%d-%m-%Y %H:%M:%S')
vaclog["Time"] = pd.to_datetime(vaclog["Time"],format='%d-%m-%Y %H:%M:%S')

vaclog.head()

,Live comments,Time,Elapsed time,100mbar inj,Barion_2,Barion_1,DUAL experiment,DUAL insulation,1mbar inj,helium,T-platinum,T-CERNOX,V_heater,DUAL inj line,I_emission,I_grid
0,opening turbo,2023-05-02 09:41:34,0.0,-0.035620,0.0,0.0,1.482000e-06,0.000002,0.010831,0.0,99.887,61.167,0.000026,2.260000e-07,NaN,NaN
1,NaN,2023-05-02 09:41:43,9.0,-0.035471,0.0,0.0,4.555000e-07,0.000002,0.010836,0.0,100.000,61.279,0.000022,2.260000e-07,NaN,NaN
2,NaN,2023-05-02 09:41:52,18.0,-0.035962,0.0,0.0,3.039000e-08,0.000002,0.010849,0.0,99.887,61.167,0.000024,2.260000e-07,NaN,NaN
3,NaN,2023-05-02 09:42:00,26.0,-0.036061,0.0,0.0,2.270000e-08,0.000002,0.010831,0.0,100.000,61.279,0.000025,2.260000e-07,NaN,NaN
4,NaN,2023-05-02 09:42:09,35.0,-0.035790,0.0,0.0,1.931000e-08,0.000002,0.010853,0.0,99.775,61.055,0.000025,2.260000e-07,NaN,NaN


In [69]:
#Temperature curve for CERNOX 
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


### Injection pressure linearity

In [70]:
""" #defining a function for the straight line
def func(x,a,b):
    return a*x + b
#curve fit for the data
y = inj["injection 100mbar"].values
x = (inj["Elapsed time"]/3600).values
params, cov = curve_fit(func,x,y)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}") """

' #defining a function for the straight line\ndef func(x,a,b):\n    return a*x + b\n#curve fit for the data\ny = inj["injection 100mbar"].values\nx = (inj["Elapsed time"]/3600).values\nparams, cov = curve_fit(func,x,y)\n#straight line parameters\na, b = params\nprint(f"Fitted line: y = {a:.2f}*x+{b:.2f}") '

In [71]:
#print hv comments of first file
print(pd.unique(df["Live comments"]))
vaclog_comm = vaclog.loc[:,["Time","Live comments"]]
vaclog_comm.dropna(inplace=True)
print(vaclog_comm)
vaclog_comm.index

['opening turbo' nan 'injection volume prepared' 'LHe transfer complete'
 'started MID scan' 'starting injection' 'closed turbo'
 'restarting injection' 'stable inj established']
                    Time              Live comments
0    2023-05-02 09:41:34              opening turbo
1869 2023-05-02 14:10:33  injection volume prepared
1933 2023-05-02 14:19:43      LHe transfer complete
1952 2023-05-02 14:22:25           started MID scan
1987 2023-05-02 14:27:28         starting injection
2026 2023-05-02 14:33:03               closed turbo
2062 2023-05-02 14:38:12       restarting injection
2085 2023-05-02 14:41:29     stable inj established


Int64Index([0, 1869, 1933, 1952, 1987, 2026, 2062, 2085], dtype='int64')

## Plotting

In [72]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")


In [73]:
#Plotting the general pressure evolution for overview
fig, ax = plt.subplots()

l5 = ax.plot(vaclog["Time"], polyfit(A,vaclog["T-CERNOX"]),marker=".", color="cyan", markersize=5,label='Temperature')
ax.set_ylabel('Temperature (K)')
ax.set_xlabel('Timestamp')
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
#comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in vaclog_comm.index:
    plt.text(vaclog_comm["Time"][i],0.8,str(vaclog_comm["Live comments"][i]),fontdict=font, transform=trans)
    


ax2=ax.twinx()

l1 = ax2.plot((vaclog["Time"]),vaclog['Barion_1']/0.46,marker=".", markersize=5,label='BA1')
l2 = ax2.plot((vaclog["Time"]),vaclog['Barion_2']/0.46,marker=".", markersize=5,label='BA2')
l3 = ax2.plot((vaclog["Time"]),vaclog['DUAL experiment'],marker=".", markersize=5,label='DUAL')
l4 = ax2.plot((vaclog["Time"]),vaclog['1mbar inj'],marker=".", markersize=5,label='1mbar inj')

ax2.set_ylabel('Pressure (mbar)')

ax2.legend(handles = l1+l2+l3+l4+l5, loc="upper left")
plt.title("S1 ads. isotherm on Cu Sample: Pressure evolution + temperatures. AS read from file")
ax2.set_yscale('log')
ax2.tick_params(axis="y", which='minor')
ax2.grid(which='minor', axis='y')
plt.savefig(r'./graphs/Barions.png')
plt.show()


""" #Plotting BA2, BA1 pressure
plt.figure(figsize=(12,6))
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S1 Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
       
plt.savefig(r'./graphs/Barions.png')
plt.show() """

' #Plotting BA2, BA1 pressure\nplt.figure(figsize=(12,6))\nplt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label=\'Barion 2\')\nplt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label=\'Barion 1\')\nplt.xlabel(\'Timestamp\')\nplt.ylabel(\'Pressure (mbar)\')\nplt.legend()\nplt.title("S1 Total pressure evolution")\nplt.tick_params(axis="y", which=\'minor\')\nplt.grid(which=\'minor\', axis=\'y\')\nplt.yscale(\'log\')\n\nax.xaxis.set_major_formatter(md.DateFormatter(\'%H:%M\'))\nax.xaxis.set_major_locator(plt.MaxNLocator(10))\nfor label in ax.get_xticklabels(which=\'major\'):\n    label.set(rotation=30, horizontalalignment=\'right\')    \n       \nplt.savefig(r\'./graphs/Barions.png\')\nplt.show() '

In [74]:
""" #checking inj pressure linearity, graph
fig, ax = plt.subplots()
y_fit = func(x,a,b)
ax.plot((inj["Elapsed time"]/3600),inj["injection 100mbar"],marker=".", markersize=5,label='inj volume pressure')
ax.plot((inj["Elapsed time"]/3600),y_fit, "--",color="red", label="Y= -ax + b fit") 
ax.legend(loc="upper right")

ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')
plt.title("Injection pressure evolution 100mbar conductance gauge") """

' #checking inj pressure linearity, graph\nfig, ax = plt.subplots()\ny_fit = func(x,a,b)\nax.plot((inj["Elapsed time"]/3600),inj["injection 100mbar"],marker=".", markersize=5,label=\'inj volume pressure\')\nax.plot((inj["Elapsed time"]/3600),y_fit, "--",color="red", label="Y= -ax + b fit") \nax.legend(loc="upper right")\n\nax.set_xlabel(\'Elapsed time (h)\')\nax.set_ylabel(\'Pressure (mbar)\')\nax.set_yscale(\'linear\')\nplt.title("Injection pressure evolution 100mbar conductance gauge") '